In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece==0.1.91
!pip install transformers==4.8.2
!pip install torch==1.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 31.2 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595750 sha256=201d34bb21ed7066c54926478d1d04c9ddf3bd7ee5f6add3fc5ff99a00a79a06
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 32.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zu37awzb
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-zu37awzb
     |████████████████████████████████| 132 kB 29.3 MB/s 
     |████████████████████████████████| 4.5 MB 24.8 MB/s 
     |████████████████████████████████| 9.0 MB 50.6 MB/s 
     |████████████████████████████████| 79 kB 8.2 MB/s 
     |████████████████████████████████| 127 kB 69.4 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=c8b6a5cb2d456775f8041d95e2707006e8aff1b5597f496779f8c66ec6be9678
  Stored in directory: /tmp/pip-ephem-wheel-cache-an8742po/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling url

In [ ]:
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
#Dataset은 샘플과 label을 저장하고, DataLoader는 dataset을 샘플에 쉽게 접근할 수 있도록 순회 가능한 객체로 감싼다
#DataLoader(데이터셋, 미니배치 크기, shuffle)
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
#CPU사용시
#device = torch.device("cpu:0")

#GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

In [ ]:
data = pd.read_excel('/content/gdrive/MyDrive/Colab Notebooks/final_data.xlsx', engine = 'openpyxl')

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size = 0.25, random_state=42, shuffle = True, stratify = data['label'])

In [ ]:
train_data = train_data[['document', 'label']]
test_data = test_data[['document', 'label']]

In [ ]:
#BERTDataset에 들어가는 데이터의 형식은 기존의 train_data, test_data와 다르다.
  #리스트 안에 리스트가 있는 형식으로 들어가야 하는 것 같다.
    #data = [['나 오늘 기분이 좋아', 1], ['오늘 날씨가 너무 안좋아서 꿀꿀하네', 0]] 이런 식으로
    #따라서 기존의 train_data, test_data를 위의 형식으로 바꿔야 한다.
train_data_list = []
for sent, label in zip(train_data['document'], train_data['label']):
  data = []
  data.append(sent)
  data.append(str(label))
  train_data_list.append(data)

In [ ]:
test_data_list = []
for sent, label in zip(test_data['document'], test_data['label']):
  data = []
  data.append(sent)
  data.append(str(label))
  test_data_list.append(data)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# Setting parameters 필수
max_len = 32
batch_size = 256
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 100
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(train_data_list, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(test_data_list, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)
#num_worker: 데이터 로딩에 사용하는 subprocess의 개수(멀티프로세싱)
  #CPU와 GPU의 trade-off관계... 

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 2, 
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
  train_acc = 0.0
  test_acc = 0.0
  model.train()
  for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
      optimizer.zero_grad()
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      loss = loss_fn(out, label)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
      optimizer.step()
      scheduler.step()  # Update learning rate schedule
      train_acc += calc_accuracy(out, label)
      if batch_id % log_interval == 0:
          print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
  print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
  model.eval()
  for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
      token_ids = token_ids.long().to(device)
      segment_ids = segment_ids.long().to(device)
      valid_length= valid_length
      label = label.long().to(device)
      out = model(token_ids, valid_length, segment_ids)
      test_acc += calc_accuracy(out, label)
  print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 1/890 [00:01<22:35,  1.53s/it]

epoch 1 batch id 1 loss 0.6596890091896057 train acc 0.58984375


 11%|█▏        | 101/890 [02:25<18:36,  1.41s/it]

epoch 1 batch id 101 loss 0.323384553194046 train acc 0.8379873143564357


 23%|██▎       | 201/890 [04:47<16:17,  1.42s/it]

...

epoch 5 batch id 101 loss 0.07639984041452408 train acc 0.9603960396039604


 23%|██▎       | 201/890 [04:46<16:19,  1.42s/it]

epoch 5 batch id 201 loss 0.06419435888528824 train acc 0.9626088308457711


 31%|███▏      | 280/890 [06:40<14:32,  1.43s/it]
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py"

KeyboardInterrupt: ignored

# KoBERT 모델 저장하기

In [ ]:
path = '/content/gdrive/MyDrive/CUAI/CUAI/하계 프로젝트/'
torch.save(model, path + 'KoBERT_감정분석.pt')  # 전체 모델 저장
torch.save(model.state_dict(), path + 'model_state_dict.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, path + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능